## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,55.53,68,96,27.02,overcast clouds
1,1,San Julian,MX,21.0167,-102.1667,68.02,47,68,7.14,broken clouds
2,2,Upernavik,GL,72.7868,-56.1549,29.95,97,100,16.11,snow
3,3,Souillac,MU,-20.5167,57.5167,71.98,83,40,3.44,scattered clouds
4,4,Garissa,KE,-0.4536,39.6401,79.18,71,74,13.31,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
#Ask the customer to add a minimum and maximum temp value
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Souillac,MU,-20.5167,57.5167,71.98,83,40,3.44,scattered clouds
4,4,Garissa,KE,-0.4536,39.6401,79.18,71,74,13.31,broken clouds
8,8,Vaini,TO,-21.2000,-175.2000,78.96,83,40,9.22,light rain
15,15,Rikitea,PF,-23.1203,-134.9692,73.22,86,73,14.99,light rain
18,18,Karratha,AU,-20.7377,116.8463,71.98,64,0,6.22,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Souillac,MU,-20.5167,57.5167,71.98,83,40,3.44,scattered clouds
4,4,Garissa,KE,-0.4536,39.6401,79.18,71,74,13.31,broken clouds
8,8,Vaini,TO,-21.2000,-175.2000,78.96,83,40,9.22,light rain
15,15,Rikitea,PF,-23.1203,-134.9692,73.22,86,73,14.99,light rain
18,18,Karratha,AU,-20.7377,116.8463,71.98,64,0,6.22,clear sky
...,...,...,...,...,...,...,...,...,...,...
666,666,Port Keats,AU,-14.2500,129.5500,84.61,59,43,5.01,scattered clouds
667,667,Carauari,BR,-4.8828,-66.8958,80.01,75,94,1.74,light rain
668,668,Sao Gabriel Da Cachoeira,BR,-0.1303,-67.0892,77.81,85,76,2.19,broken clouds
669,669,Canton,CN,23.1167,113.2500,71.01,75,97,4.36,overcast clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Souillac,MU,71.98,scattered clouds,-20.5167,57.5167,
4,Garissa,KE,79.18,broken clouds,-0.4536,39.6401,
8,Vaini,TO,78.96,light rain,-21.2000,-175.2000,
15,Rikitea,PF,73.22,light rain,-23.1203,-134.9692,
18,Karratha,AU,71.98,clear sky,-20.7377,116.8463,
25,Butaritari,KI,81.54,light rain,3.0707,172.7902,
27,Galesong,ID,75.22,overcast clouds,-5.3166,119.3661,
28,Ixtapa,MX,85.23,clear sky,20.7000,-105.2000,
29,San Patricio,US,86.09,clear sky,28.0170,-97.5169,
31,Mount Isa,AU,78.57,light rain,-20.7333,139.5000,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping") 
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Souillac,MU,71.98,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
4,Garissa,KE,79.18,broken clouds,-0.4536,39.6401,Sharif Plaza Hotel
8,Vaini,TO,78.96,light rain,-21.2000,-175.2000,Keleti Beach Resort
15,Rikitea,PF,73.22,light rain,-23.1203,-134.9692,People ThankYou
18,Karratha,AU,71.98,clear sky,-20.7377,116.8463,ibis Styles Karratha
...,...,...,...,...,...,...,...
666,Port Keats,AU,84.61,scattered clouds,-14.2500,129.5500,TDC Village (Hundred Man Camp)
667,Carauari,BR,80.01,light rain,-4.8828,-66.8958,Hotel Fellip
668,Sao Gabriel Da Cachoeira,BR,77.81,broken clouds,-0.1303,-67.0892,Alarne - Área De Lazer Do Alto Rio Negro
669,Canton,CN,71.01,overcast clouds,23.1167,113.2500,Guangzhou Hotel


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))